<a href="https://colab.research.google.com/github/meti-94/TextGeneration/blob/main/personachat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title SSH Colab Dropbear
import apt, apt.debfile
import pathlib, stat, shutil, urllib.request, subprocess, getpass, time, tempfile
import secrets, json, re
import IPython.utils.io
import ipywidgets
import socket
import os
from IPython.display import clear_output

class _NoteProgress(apt.progress.base.InstallProgress, apt.progress.base.AcquireProgress, apt.progress.base.OpProgress):
  def __init__(self):
    apt.progress.base.InstallProgress.__init__(self)
    self._label = ipywidgets.Label()
    display(self._label)
    self._float_progress = ipywidgets.FloatProgress(min = 0.0, max = 1.0, layout = {'border':'1px solid #118800'})
    display(self._float_progress)

  def close(self):
    self._float_progress.close()
    self._label.close()

  def fetch(self, item):
    self._label.value = "fetch: " + item.shortdesc

  def pulse(self, owner):
    self._float_progress.value = self.current_items / self.total_items
    return True

  def status_change(self, pkg, percent, status):
    self._label.value = "%s: %s" % (pkg, status)
    self._float_progress.value = percent / 100.0

  def update(self, percent=None):
    self._float_progress.value = self.percent / 100.0
    self._label.value = self.op + ": " + self.subop

  def done(self, item=None):
    pass

class _MyApt:
  def __init__(self):
    self._progress = _NoteProgress()
    self._cache = apt.Cache(self._progress)

  def close(self):
    self._cache.close()
    self._cache = None
    self._progress.close()
    self._progress = None

  def update_upgrade(self):
    self._cache.update()
    self._cache.open(None)
    self._cache.upgrade()

  def commit(self):
    self._cache.commit(self._progress, self._progress)
    self._cache.clear()

  def installPkg(self, *args):
    for name in args:
      pkg = self._cache[name]
      if pkg.is_installed:
        print(f"{name} is already installed")
      else:
        pkg.mark_install()

  def installDebPackage(self, name):
    apt.debfile.DebPackage(name, self._cache).install()

  def deleteInstalledPkg(self, *args):
    for pkg in self._cache:
      if pkg.is_installed:
        for name in args:
          if pkg.name.startswith(name):
            #print(f"Delete {pkg.name}")
            pkg.mark_delete()

def _download(url, path):
  try:
    with urllib.request.urlopen(url) as response:
      with open(path, 'wb') as outfile:
        shutil.copyfileobj(response, outfile)
  except:
    print("Failed to download ", url)
    raise

  return True

def _setupssh():
  #Inpur Username
  print("Create a username :")
  user_name = input()
  subprocess.run(["useradd", "-s", "/bin/bash", "-m", user_name])
  subprocess.run(["adduser", user_name, "sudo"], check = True)
  clear_output()

  #Inpur Password
  print("Create a password :")
  user_password = getpass.getpass()
  subprocess.run(["chpasswd"], input = f"{user_name}:{user_password}", universal_newlines = True)
  clear_output()

  #Set your ngrok Authtoken.
  print("Copy & paste your tunnel authtoken from https://dashboard.ngrok.com/auth")
  print("(You need to sign up for ngrok and login.)")
  ngrok_token = getpass.getpass()
  clear_output()

  #Set your ngrok region.
  print("Select your ngrok region :")
  print("us - United States (Ohio)")
  print("eu - Europe (Frankfurt)")
  print("ap - Asia/Pacific (Singapore)")
  print("au - Australia (Sydney)")
  print("sa - South America (Sao Paulo)")
  print("jp - Japan (Tokyo)")
  print("in - India (Mumbai)")
  ngrok_region = input()
  clear_output()

  #SSH Dropbear
  my_apt = _MyApt()
  my_apt.installPkg("dropbear")
  my_apt.commit()
  my_apt.close()

  #Set Dropbear
  f = open("../etc/default/dropbear", "w")
  f.write("""# the TCP port that Dropbear listens on
DROPBEAR_PORT=443

# any additional arguments for Dropbear
DROPBEAR_EXTRA_ARGS=

# specify an optional banner file containing a message to be
# sent to clients before they connect, such as "/etc/issue.net"
DROPBEAR_BANNER=""

# RSA hostkey file (default: /etc/dropbear/dropbear_rsa_host_key)
#DROPBEAR_RSAKEY="/etc/dropbear/dropbear_rsa_host_key"

# DSS hostkey file (default: /etc/dropbear/dropbear_dss_host_key)
#DROPBEAR_DSSKEY="/etc/dropbear/dropbear_dss_host_key"

# ECDSA hostkey file (default: /etc/dropbear/dropbear_ecdsa_host_key)
#DROPBEAR_ECDSAKEY="/etc/dropbear/dropbear_ecdsa_host_key"

# Receive window size - this is a tradeoff between memory and
# network performance
DROPBEAR_RECEIVE_WINDOW=65536
""")
  f.close()

  #SSH Service Restart
  subprocess.run(["service", "dropbear", "restart"])

  #Root Password
  root_password = user_password
  subprocess.run(["chpasswd"], input = f"root:{root_password}", universal_newlines = True)

  # Ngrok Tunnel
  if not os.path.exists('ngrok.zip'):
    _download("https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip", "ngrok.zip")
    shutil.unpack_archive("ngrok.zip")
    pathlib.Path("ngrok").chmod(stat.S_IXUSR)

  subprocess.run(["./ngrok", "authtoken", ngrok_token])
  ngrok_proc = subprocess.Popen(["./ngrok", "tcp", "-region", ngrok_region, "443"])
  time.sleep(2)

  with urllib.request.urlopen("http://localhost:4040/api/tunnels") as response:
    url = json.load(response)['tunnels'][0]['public_url']
    m = re.match("tcp://(.+):(\d+)", url)

  hostname = m.group(1)
  port = m.group(2)
  
  msg = ""
  msg += "="*48 + "\n"
  msg += "Command to connect to the ssh server:\n"
  msg += f"ssh -p {port} {user_name}@{socket.gethostbyname(hostname)}\n"
  msg += "="*48 + "\n"
  msg += "SSH Login NetMod and Others:\n"
  msg += f"Host            : {socket.gethostbyname(hostname)}\n"
  msg += f"Port (Dropbear) : {port}\n"
  msg += f"Username        : {user_name}\n"
  msg += f"Password        : {user_password}\n"
  msg += "="*48 + "\n"
  return msg

def startColab():
  msg = _setupssh()
  print(msg)

startColab()

Create a username :


KeyboardInterrupt: ignored

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar -xf  simple-examples.tgz
!mkdir data
!mv /content/simple-examples/data/ptb.train.txt data/
!mv /content/simple-examples/data/ptb.valid.txt data/
!mv /content/simple-examples/data/ptb.test.txt data/
!rm -rf ./simple_examples

In [1]:
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install -U ray
!pip install -U transformers datasets
!pip install tensorboardX


     |████████████████████████████████| 2.1MB 10.0MB/s 
     |████████████████████████████████| 3.3MB 37.4MB/s 
     |████████████████████████████████| 901kB 41.4MB/s 
     |████████████████████████████████| 204kB 10.9MB/s 
     |████████████████████████████████| 245kB 16.5MB/s 
     |████████████████████████████████| 112kB 21.4MB/s 
     |████████████████████████████████| 47.5MB 63kB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 133kB 41.6MB/s 
     |████████████████████████████████| 3.1MB 31.9MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 204kB 45.9MB/s 
     |████████████████████████████████| 1.3MB 33.1MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 143kB 39.3MB/s 
     |████████████████████████████████| 296kB 43.1MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-

In [2]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
!mkdir models

--2021-04-21 11:28:20--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.129.37
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.129.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  35.1MB/s    in 6.5s    

2021-04-21 11:28:27 (30.6 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import ray
from torch.utils.data import Dataset, DataLoader
from transformers import EncoderDecoderModel, BertTokenizer, BertModel, BertConfig
import torch
from transformers import AdamW
from tqdm import tqdm
import json
import random 
from sklearn.model_selection import train_test_split
from datetime import datetime
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from tqdm import tqdm_notebook as tqdm
import logging
import datasets
logging.basicConfig(level=logging.DEBUG)

In [5]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<persona>", "<speaker1>", "<speaker2>", "<pad>"]

ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>', '<persona>']}

encoder_max_length=512
decoder_max_length=128

def read_data(data_json_path='/content/personachat_self_original.json'):
  with open(data_json_path) as json_file:
    data_dict = json.load(json_file)
  return data_dict

def data_to_samples(data_dict, test=False):
  samples=[]
  for dialogue in (data_dict['train'] if test==False else data_dict['valid']):
    original_persona = dialogue['personality']
    
    for item in dialogue['utterances']:
      original_persona = [original_persona[-1]] + original_persona[:-1]
      history = item['history']
      response = item['candidates'][-1]
      samples.append({
          'persona':original_persona,
          'history':history,
          'response':response
      })
  return samples
  
def bertified(samples):
  bertified_data = []
  for item in samples:
    persona = ' <persona> '.join(item['persona'])
    persona = '<bos> ' + persona
    history = ''
    speakers = [" <speaker1> ", " <speaker2> "]
    speaker = 0
    for hst in item['history'][::-1]:
      history = speakers[speaker] + hst + history
      speaker = 1 - speaker
    response = '<speaker2> ' + item['response'] + ' <eos>'
    bertified_data.append({
          'persona':persona.replace('  ', ' '),
          'history':history.replace('  ', ' '),
          'input': persona.replace('  ', ' ')+' '+history.replace('  ', ' '), 
          'response':response.replace('  ', ' ')
      })
  return bertified_data

def bertified_to_model_food(bertified_data, tokenizer):
    model_food = []
    pbar = tqdm(bertified_data)
    pbar.set_description('Tokenizing Data Sample ...')
    for item in pbar:
        inputs = tokenizer(item['input'], add_special_tokens=True, max_length=encoder_max_length, truncation=True, padding="max_length")
        outputs = tokenizer(item['response'], add_special_tokens=True, max_length=decoder_max_length, truncation=True, padding="max_length")
        _item = {}
        _item["input_ids"] = inputs.input_ids
        _item["attention_mask"] = inputs.attention_mask
        _item["decoder_input_ids"] = outputs.input_ids
        _item["decoder_attention_mask"] = outputs.attention_mask
        _item["labels"] = outputs.input_ids.copy()
        model_food.append(_item)
    return model_food 

In [6]:
class PersonaDataset_v3(Dataset):
  '''
      Convert Data to proper Tensor dataset
  '''
  def __init__(self, samples):
    self.samples = samples
    self.n_samples = len(self.samples)

  def __getitem__(self, index):
    # returns specific item
    return self.samples[index]

  def __len__(self):
    return self.n_samples

In [7]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("prajjwal1/bert-tiny", "prajjwal1/bert-tiny")
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
model.get_encoder().resize_token_embeddings(len(tokenizer))
model.get_decoder().resize_token_embeddings(len(tokenizer))

data = read_data()
data_samples = data_to_samples(data)
bertified_data = bertified(data_samples)
train, valid = train_test_split(bertified_data[:100], test_size=0.15, random_state=99)
train = bertified_to_model_food(train, tokenizer)
valid = bertified_to_model_food(valid, tokenizer)
test_data = read_data()
test_data_samples = data_to_samples(test_data, True)
test = bertified(test_data_samples)
test = bertified_to_model_food(test[:100], tokenizer)



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139677410163472 on /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d.lock
INFO:filelock:Lock 139677410163472 acquired on /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 285


DEBUG:filelock:Attempting to release lock 139677410163472 on /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d.lock
INFO:filelock:Lock 139677410163472 released on /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 139677405003152 on /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b1602e16cf6061aaae41a082c5.038e1aed90492a59d2283f9c44c9fe3ee2380495ff1e7fefb3f1f04af3b685b5.lock
INFO:filelock:Lock 139677405003152 acquired on /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b1602e16cf6061aaae41a082c5.038e1aed90492a59d2283f9c44c9fe3ee2380495ff1e7fefb3f1f04af3b685b5.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn-lfs.huggingface.co:443
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /prajjwal1/bert-tiny/dab2c2bddcfb48ea430ef63fd76d46d67d704487844d967256a50dd7d7fd0a66 HTTP/1.1" 200 17756393


DEBUG:filelock:Attempting to release lock 139677405003152 on /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b1602e16cf6061aaae41a082c5.038e1aed90492a59d2283f9c44c9fe3ee2380495ff1e7fefb3f1f04af3b685b5.lock
INFO:filelock:Lock 139677405003152 released on /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b1602e16cf6061aaae41a082c5.038e1aed90492a59d2283f9c44c9fe3ee2380495ff1e7fefb3f1f04af3b685b5.lock


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

DEBUG:filelock:Attempting to release lock 139677404977296 on /root/.cache/huggingface/transformers/585ac1c3dedc6b808dd35e8770afafe10905d3e723a02617af749d39db780e09.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
INFO:filelock:Lock 139677404977296 released on /root/.cache/huggingface/transformers/585ac1c3dedc6b808dd35e8770afafe10905d3e723a02617af749d39db780e09.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/special_tokens_map.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/tokenizer.json HTTP/1.1" 404 0
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [8]:
# train_dataset = PersonaDataset_v3(list(zip(*map(one_line_tokenizer, tqdm(train)))))
train_dataset = PersonaDataset_v3(train[:100])
valid_dataset = PersonaDataset_v3(valid[:100])
test_dataset = PersonaDataset_v3(test[:100])



In [9]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.6.0/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /huggingface/datasets/1.6.0/metrics/rouge/rouge.py HTTP/1.1" 200 2170


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443


DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.6.0/metrics/rouge/dataset_infos.json HTTP/1.1" 404 0


In [10]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# model.config.max_length = 142
# model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [11]:
batch_size = 128
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/models/",
    do_eval=True,
    # do_train = True,
    # do_train = True,
    evaluation_strategy="steps",
    
    eval_steps=2,
    logging_steps=2,


    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    predict_with_generate=True,
    
    
    fp16=True, 
    
    save_strategy = 'steps',
    save_steps=35,
    save_total_limit = 2
    
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)
# instantiate trainer
def model_init():
    return model
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)



# trainer.train()

In [12]:
# from ray import tune
# from ray.tune.suggest.hyperopt import HyperOptSearch
# from ray.tune.schedulers import PopulationBasedTraining

trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    # n_samples=10, # number of trials
    # n_jobs=2  # number of parallel jobs, if multiple GPUs
)

# scheduler = PopulationBasedTraining(
#         time_attr="training_iteration",
#         metric="eval_acc",
#         mode="max",
#         perturbation_interval=1,
#         hyperparam_mutations={
#             "weight_decay": tune.uniform(0.0, 0.3),
#             "learning_rate": tune.uniform(1e-5, 5e-5),
#             "per_device_train_batch_size": [16, 32, 64],
#         })

# best_trial = trainer.hyperparameter_search(
#     direction="maximize",
#     keep_checkpoints_num=1,
    
#     backend="ray",
#     # Choose among many libraries:
#     # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
#     search_alg=HyperOptSearch(),
#     # Choose among schedulers:
#     # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
#     # scheduler=scheduler)
#     )

DEBUG:ray.tune.utils.util:missing a required argument: 'local_trainer'
INFO:ray.tune.ray_trial_executor:Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2021-04-21 11:30:51,572	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-21 11:30:55,269	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Memory usage on this node: 3.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_inner_2021-04-21_11-30-55
Number of trials: 1/20 (1 RUNNING)
+--------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name         | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|--------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _inner_09460_00000 | RUNNING  |       |     5.61152e-06 |                  2 |                             8 | 38.0779 |
+--------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+




(pid=510) 2021-04-21 11:30:56.737119: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=510) 2021-04-21 11:31:01,690	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=510) Traceback (most recent call last):
(pid=510)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 248, in run
(pid=510)     self._entrypoint()
(pid=510)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=510)     self._status_reporter.get_checkpoint())
(pid=510)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=510)     output = fn()
(pid=510)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 651, in _inner
(pid=510)     inner(config, checkpoint_dir=None)
(pid=510)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 644, in inn

Result for _inner_09460_00000:
  {}
  
== Status ==
Memory usage on this node: 4.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_inner_2021-04-21_11-30-55
Number of trials: 2/20 (1 ERROR, 1 PENDING)
+--------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name         | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|--------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _inner_09460_00001 | PENDING  |       |     2.91064e-05 |                  4 |                            32 | 24.3477 |
| _inner_09460_00000 | ERROR    |       |     5.61152e-06 |                  2 |                             8 | 38.0779 |
+--------------------+----------+-------+-----

(pid=509) 2021-04-21 11:31:02.969612: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=509) 2021-04-21 11:31:07,885	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=509) Traceback (most recent call last):
(pid=509)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 248, in run
(pid=509)     self._entrypoint()
(pid=509)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=509)     self._status_reporter.get_checkpoint())
(pid=509)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=509)     output = fn()
(pid=509)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 651, in _inner
(pid=509)     inner(config, checkpoint_dir=None)
(pid=509)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 644, in inn

...

(pid=978) 2021-04-21 11:33:01.652595: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=978) 2021-04-21 11:33:06,598	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=978) Traceback (most recent call last):
(pid=978)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 248, in run
(pid=978)     self._entrypoint()
(pid=978)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=978)     self._status_reporter.get_checkpoint())
(pid=978)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=978)     output = fn()
(pid=978)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 651, in _inner
(pid=978)     inner(config, checkpoint_dir=None)
(pid=978)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 644, in inn

Result for _inner_09460_00018:
  {}
  
== Status ==
Memory usage on this node: 4.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_inner_2021-04-21_11-30-55
Number of trials: 20/20 (19 ERROR, 1 PENDING)
+--------------------+----------+-------+-----------------+--------------------+-------------------------------+----------+
| Trial name         | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|--------------------+----------+-------+-----------------+--------------------+-------------------------------+----------|
| _inner_09460_00019 | PENDING  |       |     2.33596e-05 |                  4 |                             4 | 18.1659  |
| _inner_09460_00000 | ERROR    |       |     5.61152e-06 |                  2 |                             8 | 38.0779  |
| _inner_09460_00001 | ERROR    |      

(pid=1002) 2021-04-21 11:33:08.462851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=1002) 2021-04-21 11:33:13,445	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=1002) Traceback (most recent call last):
(pid=1002)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 248, in run
(pid=1002)     self._entrypoint()
(pid=1002)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=1002)     self._status_reporter.get_checkpoint())
(pid=1002)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=1002)     output = fn()
(pid=1002)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line 651, in _inner
(pid=1002)     inner(config, checkpoint_dir=None)
(pid=1002)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/function_runner.py", line

Result for _inner_09460_00019:
  {}
  
== Status ==
Memory usage on this node: 4.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_inner_2021-04-21_11-30-55
Number of trials: 20/20 (20 ERROR)
+--------------------+----------+-------+-----------------+--------------------+-------------------------------+----------+
| Trial name         | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|--------------------+----------+-------+-----------------+--------------------+-------------------------------+----------|
| _inner_09460_00000 | ERROR    |       |     5.61152e-06 |                  2 |                             8 | 38.0779  |
| _inner_09460_00001 | ERROR    |       |     2.91064e-05 |                  4 |                            32 | 24.3477  |
| _inner_09460_00002 | ERROR    |       |     2.05

TuneError: ignored